In [ ]:
import numpy as np
import matplotlib.pyplot as plt  # Add this import
from maze import Maze
from DQNAgent import DQNAgent
from rl_evaluator import RLEvaluator, run_comparison  # Add this import
import torch

# Constants
GAME_HEIGHT = 125
GAME_WIDTH = 125
NUMBER_OF_TILES = 9
SCREEN_HEIGHT = 700
SCREEN_WIDTH = 700
TILE_SIZE = GAME_HEIGHT // NUMBER_OF_TILES

# Maze layout
level = [
    "XXXXXXXXXXXXX",
    "X           X",
    "X XXX X XXX X",
    "X   X X   X X",
    "XXX X XXX X X",
    "X   X   X   X",
    "X XXX XXX X X",
    "X X   X   X X",
    "X XXX X XXX X",
    "X   X X   X X",
    "XXX XXX XXX X",
    "XP        X X",
    "XXXXXXXXXXXXX",
]

env = Maze(level, goal_pos=(1, 5), MAZE_HEIGHT=GAME_HEIGHT, MAZE_WIDTH=GAME_WIDTH, SIZE=TILE_SIZE)

# Define state and action dimensions based on your environment's needs
state_dim = 2  # Assuming the state is a 2D coordinate (row, col)
action_dim = 4  # Four actions: left, up, right, down
agent = DQNAgent(state_dim, action_dim)

# Add evaluation function
def evaluate_agent(env, agent, num_runs=5):
    total_reward = 0
    for _ in range(num_runs):
        state = env.reset_state()
        done = False
        episode_reward = 0
        while not done:
            action = agent.get_action(state, epsilon=0.0)  # No exploration
            next_state, reward, done = env.step(action)
            episode_reward += reward
            state = next_state
        total_reward += episode_reward
    average_reward = total_reward / num_runs
    return average_reward


def plot_path(agent_path, episode, max_steps=100):
    grid = np.zeros((env.number_of_tiles, env.number_of_tiles))
    for (row, col) in env.walls:
        grid[row, col] = -1  # Represent walls with -1
    for step, (row, col) in enumerate(agent_path[:max_steps]):
        grid[row, col] = step + 1  # Mark path with step number
    grid[env.goal_pos] = 10  # Mark goal with 10
    plt.figure(figsize=(env.number_of_tiles, env.number_of_tiles))  # Adjust figure size to match maze size
    plt.imshow(grid, cmap="viridis", origin="upper")
    plt.colorbar(label="Steps (0=start, 10=goal)")
    plt.title(f"Path Taken by Agent - Episode {episode}")
    plt.show()

# Initialize lists to store rewards and losses
episode_rewards = []
episode_losses = []

# Training loop
num_episodes = 100
evaluation_interval = 50  # Evaluate every 50 episodes
for episode in range(num_episodes):
    state = env.reset_state()
    done = False
    steps = 0
    cumulative_loss = 0
    episode_reward = 0
    agent_path = [state]  # Track path for visualization

    while not done:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        agent.store_transition(state, action, reward, next_state, done)
        loss = agent.train()
        cumulative_loss += loss
        episode_reward += reward
        steps += 1
        state = next_state
        agent_path.append(state)  # Add to path

    avg_loss = cumulative_loss / steps if steps > 0 else 0
    episode_rewards.append(episode_reward)  # Store total reward
    episode_losses.append(avg_loss)         # Store average loss

    # Print episode details
    print(f"Episode {episode} completed with {steps} steps, average loss {avg_loss} and total reward {episode_reward}")

    # Evaluate the agent and plot path every 50 episodes
    #if episode % evaluation_interval == 0 and episode != 0:
        #avg_reward = evaluate_agent(env, agent)
        #print(f"Evaluation after episode {episode}: Average Reward = {avg_reward}")
    if episode_reward > 50:
        plot_path(agent_path, episode)

# After training, plot rewards and losses
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(episode_rewards)
plt.title('Episode Rewards')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.subplot(1, 2, 2)
plt.plot(episode_losses)
plt.title('Average Loss per Episode')
plt.xlabel('Episode')
plt.ylabel('Loss')
plt.tight_layout()
plt.show()

e:\Programming\Projects\TrustRegionECE5523\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Episode 0 completed with 4203 steps, average loss 0.08116441433481489 and total reward -2483.5
Episode 1 completed with 233 steps, average loss 3.221300187802478 and total reward -80.5
Episode 2 completed with 459 steps, average loss 2.9100302903221063 and total reward -298.5
Episode 3 completed with 1442 steps, average loss 5.422208903339302 and total reward -1052.5
Episode 4 completed with 264 steps, average loss 2.9701837592504243 and total reward -129.5
Episode 5 completed with 714 steps, average loss 2.663781963226174 and total reward -570.5
Episode 6 completed with 803 steps, average loss 3.0974252533496984 and total reward -616.5
Episode 7 completed with 229 steps, average loss 2.925141582322433 and total reward -110.5
Episode 8 completed with 68 steps, average loss 3.5062300931004917 and total reward 46.5
Episode 9 completed with 636 steps, average loss 3.395953525630933 and total reward -492.5
Episode 10 completed with 683 steps, average loss 3.851654521193721 and total reward

KeyboardInterrupt: 

In [ ]:
print(agent_path)

[(11, 1), (11, 2), (11, 1), (11, 2), (11, 2), (11, 1), (11, 2), (11, 3), (10, 3), (9, 3), (9, 3), (9, 3), (9, 3), (10, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 3), (9, 2), (9, 1), (8, 1), (7, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (5, 1), (5, 2), (5, 3), (4, 3), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 4), (1, 4), (1, 5)]


In [ ]:
import pygame
import os
from moviepy import ImageSequenceClip

# Initialize pygame
pygame.init()

# Constants for pygame visualization
SCREEN_WIDTH = 700
SCREEN_HEIGHT = 700
TILE_SIZE = SCREEN_WIDTH // env.number_of_tiles
FPS = 5  # Frames per second for animation

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Create screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Agent Path Animation")

# Function to draw the maze
def draw_maze(env, agent_position, path=[]):
    screen.fill(BLACK)
    for row in range(env.number_of_tiles):
        for col in range(env.number_of_tiles):
            rect = pygame.Rect(col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE)
            if (row, col) in env.walls:
                pygame.draw.rect(screen, GRAY, rect)  # Walls
            elif (row, col) == env.goal_pos:
                pygame.draw.rect(screen, GREEN, rect)  # Goal
            elif (row, col) in path:
                pygame.draw.rect(screen, BLUE, rect)  # Path
            else:
                pygame.draw.rect(screen, WHITE, rect)  # Empty space
            pygame.draw.rect(screen, BLACK, rect, 1)  # Grid lines

    # Draw agent
    agent_rect = pygame.Rect(agent_position[1] * TILE_SIZE, agent_position[0] * TILE_SIZE, TILE_SIZE, TILE_SIZE)
    pygame.draw.rect(screen, RED, agent_rect)

# Export animation
def export_animation(agent_path, filename="episode_50.mp4"):
    clock = pygame.time.Clock()
    running = True
    frame_count = 0
    frame_paths = []

    for step, position in enumerate(agent_path):
        if not running:
            break
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        draw_maze(env, position, path=agent_path[:step])
        pygame.display.flip()

        # Save frames for animation
        frame_count += 1
        frame_path = f"frame_{frame_count}.png"
        pygame.image.save(screen, frame_path)
        frame_paths.append(frame_path)

        clock.tick(FPS)

    pygame.quit()

    # Create video from frames using moviepy
    clip = ImageSequenceClip(frame_paths, fps=FPS)
    clip.write_videofile(filename, fps=FPS)

    # Clean up frame files
    for frame_path in frame_paths:
        os.remove(frame_path)

# Example usage: Export animation for episode 50
# Assuming agent_path_50 is the path of the agent in episode 50
export_animation(agent_path, filename="episode_50.mp4")


pygame 2.5.0 (SDL 2.28.0, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
MoviePy - Building video episode_50.mp4.
MoviePy - Writing video episode_50.mp4



MoviePy - Done !
MoviePy - video ready episode_50.mp4


In [ ]:
current_seed = torch.initial_seed()
print(current_seed)
print(np.random.get_state()[1][0])


434516270666000
2147483648


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from maze import Maze

class replay_buffer(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = deque(maxlen=self.capacity)

    def store(self, observation, action, reward, next_observation, done):
        observation = np.expand_dims(observation, 0)
        next_observation = np.expand_dims(next_observation, 0)

        self.memory.append([observation, action, reward, next_observation, done])

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        observation, action, reward, next_observation, done = zip(* batch)

        return np.concatenate(observation, 0), action, reward, np.concatenate(next_observation, 0), done

    def __len__(self):
        return len(self.memory)


class NoisyLinear(nn.Module):
    def __init__(self, input_dim, output_dim, std_init=0.4):
        super(NoisyLinear, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.std_init = std_init

        self.weight_mu = nn.Parameter(torch.FloatTensor(self.output_dim, self.input_dim))
        self.weight_sigma = nn.Parameter(torch.FloatTensor(self.output_dim, self.input_dim))

        self.bias_mu = nn.Parameter(torch.FloatTensor(self.output_dim))
        self.bias_sigma = nn.Parameter(torch.FloatTensor(self.output_dim))

        self.register_buffer('weight_epsilon', torch.FloatTensor(self.output_dim, self.input_dim))
        self.register_buffer('bias_epsilon', torch.FloatTensor(self.output_dim))

        self.reset_parameter()
        self.reset_noise()

    def forward(self, input):
        if self.training:
            weight = self.weight_mu + self.weight_sigma.mul(self.weight_epsilon)
            bias = self.bias_mu + self.bias_sigma.mul(self.bias_epsilon)
        else:
            weight = self.weight_mu
            bias = self.bias_mu
        return F.linear(input, weight, bias)

    def _scale_noise(self, size):
        noise = torch.randn(size)
        noise = noise.sign().mul(noise.abs().sqrt())
        return noise

    def reset_parameter(self):
        mu_range = 1.0 / np.sqrt(self.input_dim)

        self.weight_mu.detach().uniform_(-mu_range, mu_range)
        self.bias_mu.detach().uniform_(-mu_range, mu_range)

        self.weight_sigma.detach().fill_(self.std_init / np.sqrt(self.input_dim))
        self.bias_sigma.detach().fill_(self.std_init / np.sqrt(self.output_dim))

    def reset_noise(self):
        epsilon_in = self._scale_noise(self.input_dim)
        epsilon_out = self._scale_noise(self.output_dim)

        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(self._scale_noise(self.output_dim))


class categorical_dqn(nn.Module):
    def __init__(self, observation_dim, action_dim, atoms_num, v_min, v_max):
        super(categorical_dqn, self).__init__()
        self.observation_dim = observation_dim
        self.action_dim = action_dim
        self.atoms_num = atoms_num
        self.v_min = v_min
        self.v_max = v_max

        self.fc1 = nn.Linear(self.observation_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.noisy1 = NoisyLinear(128, 512)
        self.noisy2 = NoisyLinear(512, self.action_dim * self.atoms_num)

    def forward(self, observation):
        x = self.fc1(observation)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.noisy1(x)
        x = F.relu(x)
        x = self.noisy2(x)
        x = F.softmax(x.view(-1, self.atoms_num), 1).view(-1, self.action_dim, self.atoms_num)
        return x

    def reset_noise(self):
        self.noisy1.reset_noise()
        self.noisy2.reset_noise()

    def act(self, observation, epsilon):
        if random.random() > epsilon:
            dist = self.forward(observation)
            dist = dist.detach()
            dist = dist.mul(torch.linspace(v_min, v_max, self.atoms_num))
            action = dist.sum(2).max(1)[1].detach()[0].item()
        else:
            action = random.choice(list(range(self.action_dim)))
        return action


def projection_distribution(target_model, next_observation, reward, done, v_min, v_max, atoms_num, gamma):
    batch_size = next_observation.size(0)
    # next_observation: [batch_size, ...]
    delta_z = float(v_max - v_min) / (atoms_num - 1)
    support = torch.linspace(v_min, v_max, atoms_num)
    # support: [atoms_num]

    next_dist = target_model.forward(next_observation).detach().mul(support)
    # next_dist: [batch_size, action_dim, atoms_num]
    next_action = next_dist.sum(2).max(1)[1]
    # next_action: [batch_size]
    next_action = next_action.unsqueeze(1).unsqueeze(1).expand(batch_size, 1, atoms_num)
    # next_action: [batch_size, 1, atoms_num]
    next_dist = next_dist.gather(1, next_action).squeeze(1)
    # next_dist: [batch_size, atoms_num]

    reward = reward.unsqueeze(1).expand_as(next_dist)
    done = done.unsqueeze(1).expand_as(next_dist)
    support = support.unsqueeze(0).expand_as(next_dist)

    Tz = reward + (1 - done) * support * gamma
    Tz = Tz.clamp(min=v_min, max=v_max)
    b = (Tz - v_min) / delta_z
    l = b.floor().long()
    u = b.ceil().long()

    offset = torch.linspace(0, (batch_size - 1) * atoms_num, batch_size).long().unsqueeze(1).expand_as(next_dist)

    proj_dist = torch.zeros_like(next_dist, dtype=torch.float32)
    proj_dist.view(-1).index_add_(0, (offset + l).view(-1), (next_dist * (u.float() - b)).view(-1))
    proj_dist.view(-1).index_add_(0, (offset + u).view(-1), (next_dist * (b - l.float())).view(-1))
    return proj_dist


def train(eval_model, target_model, buffer, v_min, v_max, atoms_num, gamma, batch_size, optimizer, count, update_freq):
    observation, action, reward, next_observation, done = buffer.sample(batch_size)

    observation = torch.FloatTensor(observation)
    action = torch.LongTensor(action)
    reward = torch.FloatTensor(reward)
    next_observation = torch.FloatTensor(next_observation)
    done = torch.FloatTensor(done)

    proj_dist = projection_distribution(target_model, next_observation, reward, done, v_min, v_max, atoms_num, gamma)

    dist = eval_model.forward(observation)
    action = action.unsqueeze(1).unsqueeze(1).expand(batch_size, 1, atoms_num)
    dist = dist.gather(1, action).squeeze(1)
    dist.detach().clamp_(0.01, 0.99)
    # * make KL divergence as the loss
    loss = - (proj_dist * dist.log()).sum(1).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    eval_model.reset_noise()
    target_model.reset_noise()

    if count % update_freq == 0:
        target_model.load_state_dict(eval_model.state_dict())
def evaluate_agent(env, agent, num_runs=5):
    total_reward = 0
    for _ in range(num_runs):
        state = env.reset_state()
        done = False
        episode_reward = 0
        while not done:
            action = agent.act(torch.FloatTensor(np.expand_dims(state, 0)), epsilon=0.0)  # No exploration
            next_state, reward, done = env.step(action)
            episode_reward += reward
            state = next_state
        total_reward += episode_reward
    average_reward = total_reward / num_runs
    return average_reward

def plot_path(agent_path, episode):
    grid = np.zeros((env.number_of_tiles, env.number_of_tiles))
    for (row, col) in env.walls:
        grid[row, col] = -1  # Represent walls with -1
    for step, (row, col) in enumerate(agent_path):
        grid[row, col] = step + 1  # Mark path with step number
    grid[env.goal_pos] = 10  # Mark goal with 10
    plt.imshow(grid, cmap="viridis", origin="upper")
    plt.colorbar(label="Steps (0=start, 10=goal)")
    plt.title(f"Path Taken by Agent - Episode {episode}")
    plt.show()

if __name__ == '__main__':
    episode = 100
    epsilon_init = 0.95
    epsilon_decay = 0.95
    epsilon_min = 0.01
    update_freq = 10
    gamma = 0.99
    learning_rate = 1e-3
    atoms_num = 51
    v_min = -100
    v_max = 100
    batch_size = 64
    capacity = 10000
    exploration = 10
    render = False
    GAME_HEIGHT = 125
    GAME_WIDTH = 125
    NUMBER_OF_TILES = 9
    SCREEN_HEIGHT = 700
    SCREEN_WIDTH = 700
    TILE_SIZE = GAME_HEIGHT // NUMBER_OF_TILES

    # Initialize the maze environment
    level = [
        "XXXXXXXXXXXXX",
        "X           X",
        "X XXX X XXX X",
        "X   X X   X X",
        "XXX X XXX X X",
        "X   X   X   X",
        "X XXX XXX X X",
        "X X   X   X X",
        "X XXX X XXX X",
        "X   X X   X X",
        "XXX XXX XXX X",
        "XP        X X",
        "XXXXXXXXXXXXX",
    ]
    env = Maze(level, goal_pos=(1, 5), MAZE_HEIGHT=GAME_HEIGHT, MAZE_WIDTH=GAME_WIDTH, SIZE=TILE_SIZE)

    action_dim = 4  # Four actions: left, up, right, down
    observation_dim = 2  # Assuming the state is a 2D coordinate (row, col)
    count = 0
    target_net = categorical_dqn(observation_dim, action_dim, atoms_num, v_min, v_max)
    eval_net = categorical_dqn(observation_dim, action_dim, atoms_num, v_min, v_max)
    target_net.load_state_dict(eval_net.state_dict())
    optimizer = torch.optim.Adam(eval_net.parameters(), lr=learning_rate)
    buffer = replay_buffer(capacity)
    weight_reward = None
    epsilon = epsilon_init

    # Initialize lists to store rewards and losses
    episode_rewards = []
    episode_losses = []

    for i in range(episode):
        obs = env.reset_state()
        reward_total = 0
        steps = 0
        cumulative_loss = 0
        agent_path = [obs]  # Track path for visualization
        if render:
            env.render()
        while True:
            action = eval_net.act(torch.FloatTensor(np.expand_dims(obs, 0)), epsilon)
            next_obs, reward, done = env.step(action)
            count += 1
            if render:
                env.render()
            buffer.store(obs, action, reward, next_obs, done)
            reward_total += reward
            obs = next_obs
            if i > exploration:
                loss = train(eval_net, target_net, buffer, v_min, v_max, atoms_num, gamma, batch_size, optimizer, count, update_freq)
                cumulative_loss += loss
            steps += 1
            agent_path.append(obs)  # Add to path
            if done:
                if epsilon > epsilon_min:
                    epsilon = epsilon * epsilon_decay
                if not weight_reward:
                    weight_reward = reward_total
                else:
                    weight_reward = 0.99 * weight_reward + 0.01 * reward_total
                avg_loss = cumulative_loss / steps if steps > 0 else 0
                episode_rewards.append(reward_total)  # Store total reward
                episode_losses.append(avg_loss)       # Store average loss
                print(f'episode: {i+1}  reward: {reward_total}  weight_reward: {weight_reward:.3f}  epsilon: {epsilon:.2f}')
                break

        # Evaluate the agent and plot path every 100 episodes if reward is more than -100
        if i % 100 == 0 and i != 0:
            avg_reward = evaluate_agent(env, eval_net)
            print(f"Evaluation after episode {i}: Average Reward = {avg_reward}")
            if reward_total > -100:
                plot_path(agent_path, i)

    # After training, plot rewards and losses
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(episode_rewards)
    plt.title('Episode Rewards')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.subplot(1, 2, 2)
    plt.plot(episode_losses)
    plt.title('Average Loss per Episode')
    plt.xlabel('Episode')
    plt.ylabel('Loss')
    plt.tight_layout()
    plt.show()